In [3]:
from __future__ import division
import numpy as np
import pyopencl as cl
import pyopencl.array

ImportError: No module named pyopencl

Load the PyOpenCL IPython extension:

In [3]:
%load_ext pyopencl.ipython_ext

Create an OpenCL context and a command queue:

In [5]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'AMD Accelerated Parallel Processing' at 0x7fc14f1b0080>
[1] <pyopencl.Platform 'Intel(R) OpenCL' at 0x32aed00>
Choice [0]:0
Set the environment variable PYOPENCL_CTX='0' to avoid being asked again.


-----

Define an OpenCL kernel using the `%%cl_kernel` magic:

In [6]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void sum_vector(__global const float *a,
__global const float *b, __global float *c)
{
  int gid = get_global_id(0);
  c[gid] = a[gid] + b[gid];
}

This looks for `cl_ctx` or `ctx` in the user namespace to find a PyOpenCL context.

Kernel names are automatically injected into the user namespace, so we can just use `sum_vector` from Python below.

Now create some data to work on:

In [7]:
n = 10000

a = cl.array.empty(queue, n, dtype=np.float32)
a.fill(15)

b_host = np.random.randn(n).astype(np.float32)
b = cl.array.to_device(queue, b_host)

c = cl.array.empty_like(a)

Run the kernel:

In [8]:
sum_vector(queue, (n,), None, a.data, b.data, c.data)

Check the result using `numpy` operations:

In [9]:
assert (c.get() == b_host + 15).all()